<a href="https://colab.research.google.com/github/nibaskumar93n-debug/Morphoinformatics/blob/main/Druggable_pocket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install py3Dmol biopython requests -q

import requests
import py3Dmol
from collections import Counter

uniprot_id = "A0A5M5DBX0"

pocket_residues = [27, 29, 30, 31, 33, 49, 52, 53, 54, 55, 56, 57, 58, 59,
                   78, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 95,
                   110, 112, 115, 146, 149, 150, 285, 287]

# Fetch using AlphaFold API
print(f"Fetching AlphaFold structure for {uniprot_id}...")

api_url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    pdb_url = data[0]['pdbUrl']

    # Download structure
    pdb_response = requests.get(pdb_url)
    pdb_file = f"{uniprot_id}.pdb"

    with open(pdb_file, 'wb') as f:
        f.write(pdb_response.content)

    print(f"✅ Successfully downloaded: {pdb_file}\n")

    # Read and visualize
    with open(pdb_file, 'r') as f:
        pdb_data = f.read()

    # Create 3D visualization
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')

    # Show protein
    view.setStyle({'cartoon': {'color': 'lightgray'}})

    # Highlight pocket
    view.addSurface(py3Dmol.VDW,
                    {'opacity': 0.7, 'color': 'cyan'},
                    {'resi': pocket_residues})

    # Color residues by type
    view.addStyle({'resi': pocket_residues, 'resn': ['VAL', 'LEU', 'ILE', 'PHE', 'TRP', 'MET', 'ALA', 'PRO']},
                  {'stick': {'color': 'yellow', 'radius': 0.3}})

    view.addStyle({'resi': pocket_residues, 'resn': ['LYS', 'ARG', 'HIS']},
                  {'stick': {'color': 'blue', 'radius': 0.3}})

    view.addStyle({'resi': pocket_residues, 'resn': ['ASP', 'GLU']},
                  {'stick': {'color': 'red', 'radius': 0.3}})

    view.addStyle({'resi': pocket_residues, 'resn': ['SER', 'THR', 'ASN', 'GLN', 'CYS', 'TYR', 'GLY']},
                  {'stick': {'color': 'green', 'radius': 0.3}})

    view.zoomTo({'resi': pocket_residues})
    view.setBackgroundColor('white')

    print("="*60)
    print("3D VISUALIZATION")
    print("="*60)
    print("🟡 Yellow = Hydrophobic")
    print("🔵 Blue = Positive charged")
    print("🔴 Red = Negative charged")
    print("🟢 Green = Polar")
    print("="*60)

    view.show()
else:
    print(f"❌ Failed (Status: {response.status_code})")


Fetching AlphaFold structure for A0A5M5DBX0...
✅ Successfully downloaded: A0A5M5DBX0.pdb

3D VISUALIZATION
🟡 Yellow = Hydrophobic
🔵 Blue = Positive charged
🔴 Red = Negative charged
🟢 Green = Polar


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
!pip install py3Dmol biopython requests -q

import requests
import py3Dmol
from collections import Counter
from Bio.PDB import PDBParser

uniprot_id = "A0A5M5DBX0"

pocket_residues = [27, 29, 30, 31, 33, 49, 52, 53, 54, 55, 56, 57, 58, 59,
                   78, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 95,
                   110, 112, 115, 146, 149, 150, 285, 287]

# Fetch using AlphaFold API
print(f"Fetching AlphaFold structure for {uniprot_id}...")

api_url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    pdb_url = data[0]['pdbUrl']

    # Download structure
    pdb_response = requests.get(pdb_url)
    pdb_file = f"{uniprot_id}.pdb"

    with open(pdb_file, 'wb') as f:
        f.write(pdb_response.content)

    print(f"✅ Successfully downloaded: {pdb_file}\n")

    # ========================================
    # EXTRACT POCKET NAME AND DETAILS
    # ========================================

    # Parse the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(uniprot_id, pdb_file)

    print("="*60)
    print("POCKET IDENTIFICATION AND NAMING")
    print("="*60)

    # Get protein information from UniProt
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    uniprot_response = requests.get(uniprot_url)

    if uniprot_response.status_code == 200:
        uniprot_data = uniprot_response.json()

        # Get protein name
        protein_name = "Unknown"
        if 'proteinDescription' in uniprot_data:
            if 'recommendedName' in uniprot_data['proteinDescription']:
                protein_name = uniprot_data['proteinDescription']['recommendedName']['fullName']['value']
            elif 'submittedName' in uniprot_data['proteinDescription']:
                protein_name = uniprot_data['proteinDescription']['submittedName'][0]['fullName']['value']

        organism = uniprot_data.get('organism', {}).get('scientificName', 'Unknown')

        print(f"Protein: {protein_name}")
        print(f"Organism: {organism}")
        print(f"UniProt ID: {uniprot_id}")

    # Extract residue details from pocket
    pocket_residue_details = []

    for model in structure:
        for chain in model:
            chain_id = chain.get_id()
            for residue in chain:
                res_num = residue.get_id()[1]
                if res_num in pocket_residues:
                    res_name = residue.get_resname()
                    pocket_residue_details.append({
                        'chain': chain_id,
                        'resnum': res_num,
                        'resname': res_name
                    })

    # Sort by residue number
    pocket_residue_details.sort(key=lambda x: x['resnum'])

    # Generate pocket name based on characteristics
    residue_names = [r['resname'] for r in pocket_residue_details]

    # Classify residues
    hydrophobic = ['VAL', 'LEU', 'ILE', 'PHE', 'TRP', 'MET', 'ALA', 'PRO']
    positive = ['LYS', 'ARG', 'HIS']
    negative = ['ASP', 'GLU']
    polar = ['SER', 'THR', 'ASN', 'GLN', 'CYS', 'TYR', 'GLY']
    aromatic = ['PHE', 'TYR', 'TRP', 'HIS']

    hydrophobic_count = sum(1 for r in residue_names if r in hydrophobic)
    positive_count = sum(1 for r in residue_names if r in positive)
    negative_count = sum(1 for r in residue_names if r in negative)
    polar_count = sum(1 for r in residue_names if r in polar)
    aromatic_count = sum(1 for r in residue_names if r in aromatic)

    total = len(residue_names)

    # Determine pocket type
    pocket_type = []
    if hydrophobic_count / total > 0.4:
        pocket_type.append("Hydrophobic")
    if aromatic_count >= 3:
        pocket_type.append("Aromatic")
    if positive_count > negative_count + 1:
        pocket_type.append("Cationic")
    elif negative_count > positive_count + 1:
        pocket_type.append("Anionic")
    if polar_count / total > 0.3:
        pocket_type.append("Polar")

    if not pocket_type:
        pocket_type = ["Mixed"]

    # Generate pocket name
    pocket_name = f"{'-'.join(pocket_type)} Binding Pocket"

    print(f"\n🔹 POCKET NAME: {pocket_name}")
    print(f"🔹 Location: Residues {min(pocket_residues)}-{max(pocket_residues)}")
    print(f"🔹 Size: {total} residues")
    print(f"🔹 Chain: {pocket_residue_details[0]['chain']}")

    # Alternative naming schemes
    print(f"\n--- Alternative Names ---")
    print(f"Descriptive: '{protein_name[:30]}... Pocket-1'")
    print(f"Residue-based: 'Pocket_{min(pocket_residues)}_{max(pocket_residues)}'")
    print(f"Characteristic: '{pocket_name}'")

    print(f"\n--- Pocket Composition ---")
    print(f"Hydrophobic: {hydrophobic_count} ({hydrophobic_count/total*100:.1f}%)")
    print(f"Positive:    {positive_count} ({positive_count/total*100:.1f}%)")
    print(f"Negative:    {negative_count} ({negative_count/total*100:.1f}%)")
    print(f"Polar:       {polar_count} ({polar_count/total*100:.1f}%)")
    print(f"Aromatic:    {aromatic_count} ({aromatic_count/total*100:.1f}%)")

    # Key residues
    print(f"\n--- Key Residues ---")
    res_counter = Counter(residue_names)
    for res, count in res_counter.most_common(5):
        print(f"  {res}: {count}x")

    # Residue list
    print(f"\n--- Complete Residue List ---")
    residue_list = [f"{r['resname']}{r['resnum']}" for r in pocket_residue_details]
    print(f"Residues: {', '.join(residue_list[:10])}{'...' if len(residue_list) > 10 else ''}")

    print("="*60)

    # Read and visualize
    with open(pdb_file, 'r') as f:
        pdb_data = f.read()

    # Create 3D visualization
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')

    # Show protein
    view.setStyle({'cartoon': {'color': 'lightgray'}})

    # Highlight pocket
    view.addSurface(py3Dmol.VDW,
                    {'opacity': 0.7, 'color': 'cyan'},
                    {'resi': pocket_residues})

    # Color residues by type
    view.addStyle({'resi': pocket_residues, 'resn': ['VAL', 'LEU', 'ILE', 'PHE', 'TRP', 'MET', 'ALA', 'PRO']},
                  {'stick': {'color': 'yellow', 'radius': 0.3}})

    view.addStyle({'resi': pocket_residues, 'resn': ['LYS', 'ARG', 'HIS']},
                  {'stick': {'color': 'blue', 'radius': 0.3}})

    view.addStyle({'resi': pocket_residues, 'resn': ['ASP', 'GLU']},
                  {'stick': {'color': 'red', 'radius': 0.3}})

    view.addStyle({'resi': pocket_residues, 'resn': ['SER', 'THR', 'ASN', 'GLN', 'CYS', 'TYR', 'GLY']},
                  {'stick': {'color': 'green', 'radius': 0.3}})

    # Add label with pocket name
    view.addLabel(pocket_name,
                  {'fontColor': 'black', 'fontSize': 16, 'backgroundColor': 'white', 'backgroundOpacity': 0.8},
                  {'resi': pocket_residues[0]})

    view.zoomTo({'resi': pocket_residues})
    view.setBackgroundColor('white')

    print("\n3D VISUALIZATION")
    print("="*60)
    print("🟡 Yellow = Hydrophobic")
    print("🔵 Blue = Positive charged")
    print("🔴 Red = Negative charged")
    print("🟢 Green = Polar")
    print("="*60)

    view.show()

else:
    print(f"❌ Failed (Status: {response.status_code})")

Fetching AlphaFold structure for A0A5M5DBX0...
✅ Successfully downloaded: A0A5M5DBX0.pdb

POCKET IDENTIFICATION AND NAMING
Protein: Riboflavin biosynthesis protein RibD
Organism: Bacteroides ovatus
UniProt ID: A0A5M5DBX0

🔹 POCKET NAME: Hydrophobic-Aromatic Binding Pocket
🔹 Location: Residues 27-287
🔹 Size: 36 residues
🔹 Chain: A

--- Alternative Names ---
Descriptive: 'Riboflavin biosynthesis protei... Pocket-1'
Residue-based: 'Pocket_27_287'
Characteristic: 'Hydrophobic-Aromatic Binding Pocket'

--- Pocket Composition ---
Hydrophobic: 17 (47.2%)
Positive:    5 (13.9%)
Negative:    4 (11.1%)
Polar:       10 (27.8%)
Aromatic:    7 (19.4%)

--- Key Residues ---
  VAL: 5x
  PRO: 5x
  HIS: 3x
  CYS: 3x
  GLU: 3x

--- Complete Residue List ---
Residues: VAL27, PRO29, ASN30, PRO31, VAL33, HIS49, CYS52, GLY53, GLU54, ALA55...

3D VISUALIZATION
🟡 Yellow = Hydrophobic
🔵 Blue = Positive charged
🔴 Red = Negative charged
🟢 Green = Polar


3Dmol.js failed to load for some reason. Please check your browser console for error messages.